## About this Notebook

This notebook is going to evaluate local agents by comparing each agent with others for x times.

This notebook also includes 1) multiprocess using ray library; 2) multiprocess using multiprocessing library; 3) without multiprocess.

### Updates

**12/13/2020**: Added individual level's win-loss records.

**01/10/2021**: Added function to check results between two agents.

## Writing some popular agents

Agents are from https://www.kaggle.com/ihelon/rock-paper-scissors-agents-comparison

In [ ]:
%%writefile reactionary.py

import random
from kaggle_environments.envs.rps.utils import get_score

last_react_action = None


def reactionary(observation, configuration):
    global last_react_action
    if observation.step == 0:
        last_react_action = random.randrange(0, configuration.signs)
    elif get_score(last_react_action, observation.lastOpponentAction) <= 1:
        last_react_action = (observation.lastOpponentAction + 1) % configuration.signs

    return last_react_action

In [ ]:
%%writefile counter_reactionary.py

import random
from kaggle_environments.envs.rps.utils import get_score

last_counter_action = None


def counter_reactionary(observation, configuration):
    global last_counter_action
    if observation.step == 0:
        last_counter_action = random.randrange(0, configuration.signs)
    elif get_score(last_counter_action, observation.lastOpponentAction) == 1:
        last_counter_action = (last_counter_action + 2) % configuration.signs
    else:
        last_counter_action = (observation.lastOpponentAction + 1) % configuration.signs

    return last_counter_action

In [ ]:
%%writefile statistical.py

import random
from kaggle_environments.envs.rps.utils import get_score

action_histogram = {}

def statistical(observation, configuration):
    global action_histogram
    if observation.step == 0:
        action_histogram = {}
        return
    action = observation.lastOpponentAction
    if action not in action_histogram:
        action_histogram[action] = 0
    action_histogram[action] += 1
    mode_action = None
    mode_action_count = None
    for k, v in action_histogram.items():
        if mode_action_count is None or v > mode_action_count:
            mode_action = k
            mode_action_count = v
            continue

    return (mode_action + 1) % configuration.signs

In [ ]:
%%writefile nash_equilibrium.py

import random

def nash_equilibrium(observation, configuration):
    return random.randint(0, 2)

In [ ]:
%%writefile markov_agent.py

import numpy as np
import collections

def markov_agent(observation, configuration):
    k = 2
    global table, action_seq
    if observation.step % 250 == 0: # refresh table every 250 steps
        action_seq, table = [], collections.defaultdict(lambda: [1, 1, 1])    
    if len(action_seq) <= 2 * k + 1:
        action = int(np.random.randint(3))
        if observation.step > 0:
            action_seq.extend([observation.lastOpponentAction, action])
        else:
            action_seq.append(action)
        return action
    # update table
    key = ''.join([str(a) for a in action_seq[:-1]])
    table[key][observation.lastOpponentAction] += 1
    # update action seq
    action_seq[:-2] = action_seq[2:]
    action_seq[-2] = observation.lastOpponentAction
    # predict opponent next move
    key = ''.join([str(a) for a in action_seq[:-1]])
    if observation.step < 500:
        next_opponent_action_pred = np.argmax(table[key])
    else:
        scores = np.array(table[key])
        next_opponent_action_pred = np.random.choice(3, p=scores/scores.sum()) # add stochasticity for second part of the game
    # make an action
    action = (next_opponent_action_pred + 1) % 3
    # if high probability to lose -> let's surprise our opponent with sudden change of our strategy
    if observation.step > 900:
        action = next_opponent_action_pred
    action_seq[-1] = action
    return int(action)

In [ ]:
%%writefile memory_patterns.py

import random

def find_pattern(memory_patterns, memory, memory_length):
    """ find appropriate pattern in memory """
    for pattern in memory_patterns:
        actions_matched = 0
        for i in range(memory_length):
            if pattern["actions"][i] == memory[i]:
                actions_matched += 1
            else:
                break
        # if memory fits this pattern
        if actions_matched == memory_length:
            return pattern
    # appropriate pattern not found
    return None

def get_step_result_for_my_agent(my_agent_action, opp_action):
    """ 
        get result of the step for my_agent
        1, 0 and -1 representing win, tie and lost results of the game respectively
    """
    if my_agent_action == opp_action:
        return 0
    elif (my_agent_action == (opp_action + 1)) or (my_agent_action == 0 and opp_action == 2):
        return 1
    else:
        return -1


# maximum steps in the pattern
steps_max = 3
# minimum steps in the pattern
steps_min = 3
# maximum amount of steps until reassessment of effectiveness of current memory patterns
max_steps_until_memory_reassessment = random.randint(80, 120)

# current memory of the agent
current_memory = []
# list of 1, 0 and -1 representing win, tie and lost results of the game respectively
# length is max_steps_until_memory_reassessment
results = []
# current best sum of results
best_sum_of_results = 0
# how many times each action was performed by opponent
opponent_actions_count = [0, 0, 0]
# memory length of patterns in first group
# steps_max is multiplied by 2 to consider both my_agent's and opponent's actions
group_memory_length = steps_max * 2
# list of groups of memory patterns
groups_of_memory_patterns = []
for i in range(steps_max, steps_min - 1, -1):
    groups_of_memory_patterns.append({
        # how many steps in a row are in the pattern
        "memory_length": group_memory_length,
        # list of memory patterns
        "memory_patterns": []
    })
    group_memory_length -= 2
    

def my_agent(obs, conf):
    """ your ad here """
    global results
    global best_sum_of_results
    # action of my_agent
    my_action = None
    
    # if it's not first step, add opponent's last action to agent's current memory
    # and reassess effectiveness of current memory patterns
    if obs["step"] > 0:
        # count opponent's actions
        opponent_actions_count[obs["lastOpponentAction"]] += 1
        # add opponent's last step to current_memory
        current_memory.append(obs["lastOpponentAction"])
        # previous step won or lost
        results.append(get_step_result_for_my_agent(current_memory[-2], current_memory[-1]))
        
        # if there is enough steps added to results for memery reassessment
        if len(results) == max_steps_until_memory_reassessment:
            results_sum = sum(results)
            # if effectiveness of current memory patterns has decreased significantly
            if results_sum < (best_sum_of_results * 0.5):
                # flush all current memory patterns
                best_sum_of_results = 0
                results = []
                for group in groups_of_memory_patterns:
                    group["memory_patterns"] = []
            else:
                # if effectiveness of current memory patterns has increased
                if results_sum > best_sum_of_results:
                    best_sum_of_results = results_sum
                del results[:1]
    
    # search for my_action in memory patterns
    for group in groups_of_memory_patterns:
        # if length of current memory is bigger than necessary for a new memory pattern
        if len(current_memory) > group["memory_length"]:
            # get momory of the previous step
            previous_step_memory = current_memory[:group["memory_length"]]
            previous_pattern = find_pattern(group["memory_patterns"], previous_step_memory, group["memory_length"])
            if previous_pattern == None:
                previous_pattern = {
                    "actions": previous_step_memory.copy(),
                    "opp_next_actions": [
                        {"action": 0, "amount": 0, "response": 1},
                        {"action": 1, "amount": 0, "response": 2},
                        {"action": 2, "amount": 0, "response": 0}
                    ]
                }
                group["memory_patterns"].append(previous_pattern)
            # if such pattern already exists
            for action in previous_pattern["opp_next_actions"]:
                if action["action"] == obs["lastOpponentAction"]:
                    action["amount"] += 1
            # delete first two elements in current memory (actions of the oldest step in current memory)
            del current_memory[:2]
            
            # if action was not yet found
            if my_action == None:
                pattern = find_pattern(group["memory_patterns"], current_memory, group["memory_length"])
                # if appropriate pattern is found
                if pattern != None:
                    my_action_amount = 0
                    for action in pattern["opp_next_actions"]:
                        # if this opponent's action occurred more times than currently chosen action
                        # or, if it occured the same amount of times and this one is choosen randomly among them
                        if (action["amount"] > my_action_amount or
                                (action["amount"] == my_action_amount and random.random() > 0.5)):
                            my_action_amount = action["amount"]
                            my_action = action["response"]
    
    # if no action was found
    if my_action == None:
        # choose action randomly
        my_action = random.randint(0, 2)
    
    current_memory.append(my_action)
    return my_action

In [ ]:
%%writefile multi_armed_bandit.py


import pandas as pd
import numpy as np
import json


# base class for all agents, random agent
class agent():
    def initial_step(self):
        return np.random.randint(3)
    
    def history_step(self, history):
        return np.random.randint(3)
    
    def step(self, history):
        if len(history) == 0:
            return int(self.initial_step())
        else:
            return int(self.history_step(history))
    
# agent that returns (previousCompetitorStep + shift) % 3
class mirror_shift(agent):
    def __init__(self, shift=0):
        self.shift = shift
    
    def history_step(self, history):
        return (history[-1]['competitorStep'] + self.shift) % 3
    
    
# agent that returns (previousPlayerStep + shift) % 3
class self_shift(agent):
    def __init__(self, shift=0):
        self.shift = shift
    
    def history_step(self, history):
        return (history[-1]['step'] + self.shift) % 3    


# agent that beats the most popular step of competitor
class popular_beater(agent):
    def history_step(self, history):
        counts = np.bincount([x['competitorStep'] for x in history])
        return (int(np.argmax(counts)) + 1) % 3

    
# agent that beats the agent that beats the most popular step of competitor
class anti_popular_beater(agent):
    def history_step(self, history):
        counts = np.bincount([x['step'] for x in history])
        return (int(np.argmax(counts)) + 2) % 3
    
    
# simple transition matrix: previous step -> next step
class transition_matrix(agent):
    def __init__(self, deterministic = False, counter_strategy = False, init_value = 0.1, decay = 1):
        self.deterministic = deterministic
        self.counter_strategy = counter_strategy
        if counter_strategy:
            self.step_type = 'step' 
        else:
            self.step_type = 'competitorStep'
        self.init_value = init_value
        self.decay = decay
        
    def history_step(self, history):
        matrix = np.zeros((3,3)) + self.init_value
        for i in range(len(history) - 1):
            matrix = (matrix - self.init_value) / self.decay + self.init_value
            matrix[int(history[i][self.step_type]), int(history[i+1][self.step_type])] += 1

        if  self.deterministic:
            step = np.argmax(matrix[int(history[-1][self.step_type])])
        else:
            step = np.random.choice([0,1,2], p = matrix[int(history[-1][self.step_type])]/matrix[int(history[-1][self.step_type])].sum())
        
        if self.counter_strategy:
            # we predict our step using transition matrix (as competitor can do) and beat probable competitor step
            return (step + 2) % 3 
        else:
            # we just predict competitors step and beat it
            return (step + 1) % 3
    

# similar to the transition matrix but rely on both previous steps
class transition_tensor(agent):
    
    def __init__(self, deterministic = False, counter_strategy = False, init_value = 0.1, decay = 1):
        self.deterministic = deterministic
        self.counter_strategy = counter_strategy
        if counter_strategy:
            self.step_type1 = 'step' 
            self.step_type2 = 'competitorStep'
        else:
            self.step_type2 = 'step' 
            self.step_type1 = 'competitorStep'
        self.init_value = init_value
        self.decay = decay
        
    def history_step(self, history):
        matrix = np.zeros((3,3, 3)) + 0.1
        for i in range(len(history) - 1):
            matrix = (matrix - self.init_value) / self.decay + self.init_value
            matrix[int(history[i][self.step_type1]), int(history[i][self.step_type2]), int(history[i+1][self.step_type1])] += 1

        if  self.deterministic:
            step = np.argmax(matrix[int(history[-1][self.step_type1]), int(history[-1][self.step_type2])])
        else:
            step = np.random.choice([0,1,2], p = matrix[int(history[-1][self.step_type1]), int(history[-1][self.step_type2])]/matrix[int(history[-1][self.step_type1]), int(history[-1][self.step_type2])].sum())
        
        if self.counter_strategy:
            # we predict our step using transition matrix (as competitor can do) and beat probable competitor step
            return (step + 2) % 3 
        else:
            # we just predict competitors step and beat it
            return (step + 1) % 3

        
# looks for the same pattern in history and returns the best answer to the most possible counter strategy
class pattern_matching(agent):
    def __init__(self, steps = 3, deterministic = False, counter_strategy = False, init_value = 0.1, decay = 1):
        self.deterministic = deterministic
        self.counter_strategy = counter_strategy
        if counter_strategy:
            self.step_type = 'step' 
        else:
            self.step_type = 'competitorStep'
        self.init_value = init_value
        self.decay = decay
        self.steps = steps
        
    def history_step(self, history):
        if len(history) < self.steps + 1:
            return self.initial_step()
        
        next_step_count = np.zeros(3) + self.init_value
        pattern = [history[i][self.step_type] for i in range(- self.steps, 0)]
        
        for i in range(len(history) - self.steps):
            next_step_count = (next_step_count - self.init_value)/self.decay + self.init_value
            current_pattern = [history[j][self.step_type] for j in range(i, i + self.steps)]
            if np.sum([pattern[j] == current_pattern[j] for j in range(self.steps)]) == self.steps:
                next_step_count[history[i + self.steps][self.step_type]] += 1
        
        if next_step_count.max() == self.init_value:
            return self.initial_step()
        
        if  self.deterministic:
            step = np.argmax(next_step_count)
        else:
            step = np.random.choice([0,1,2], p = next_step_count/next_step_count.sum())
        
        if self.counter_strategy:
            # we predict our step using transition matrix (as competitor can do) and beat probable competitor step
            return (step + 2) % 3 
        else:
            # we just predict competitors step and beat it
            return (step + 1) % 3
        
# if we add all agents the algorithm will spend more that 1 second on turn and will be invalidated
# right now the agens are non optimal and the same computeations are repeated a lot of times
# the approach can be optimised to run much faster
agents = {
    'mirror_0': mirror_shift(0),
    'mirror_1': mirror_shift(1),  
    'mirror_2': mirror_shift(2),
    'self_0': self_shift(0),
    'self_1': self_shift(1),  
    'self_2': self_shift(2),
    'popular_beater': popular_beater(),
    'anti_popular_beater': anti_popular_beater(),
    'random_transitison_matrix': transition_matrix(False, False),
    'determenistic_transitison_matrix': transition_matrix(True, False),
    'random_self_trans_matrix': transition_matrix(False, True),
    'determenistic_self_trans_matrix': transition_matrix(True, True),
    'random_transitison_tensor': transition_tensor(False, False),
    'determenistic_transitison_tensor': transition_tensor(True, False),
    'random_self_trans_tensor': transition_tensor(False, True),
    'determenistic_self_trans_tensor': transition_tensor(True, True),
    
    'random_transitison_matrix_decay': transition_matrix(False, False, decay = 1.05),
    'random_self_trans_matrix_decay': transition_matrix(False, True, decay = 1.05),
    'random_transitison_tensor_decay': transition_tensor(False, False, decay = 1.05),
    'random_self_trans_tensor_decay': transition_tensor(False, True, decay = 1.05),
    
    'determenistic_transitison_matrix_decay': transition_matrix(True, False, decay = 1.05),
    'determenistic_self_trans_matrix_decay': transition_matrix(True, True, decay = 1.05),
    'determenistic_transitison_tensor_decay': transition_tensor(True, False, decay = 1.05),
    'determenistic_self_trans_tensor_decay': transition_tensor(True, True, decay = 1.05),
    
#     'random_transitison_matrix_decay2': transition_matrix(False, False, decay = 1.001),
#     'random_self_trans_matrix_decay2': transition_matrix(False, True, decay = 1.001),
#     'random_transitison_tensor_decay2': transition_tensor(False, False, decay = 1.001),
#     'random_self_trans_tensor_decay2': transition_tensor(False, True, decay = 1.001),
    
#     'determenistic_transitison_matrix_decay2': transition_matrix(True, False, decay = 1.001),
#     'determenistic_self_trans_matrix_decay2': transition_matrix(True, True, decay = 1.001),
#     'determenistic_transitison_tensor_decay2': transition_tensor(True, False, decay = 1.001),
#     'determenistic_self_trans_tensor_decay2': transition_tensor(True, True, decay = 1.001),
    
#     'random_pattern_matching_decay_1': pattern_matching(1, False, False, decay = 1.001),
#     'random_self_pattern_matching_decay_1': pattern_matching(1, False, True, decay = 1.001),
#     'determenistic_pattern_matching_decay_1': pattern_matching(1, True, False, decay = 1.001),
#     'determenistic_self_pattern_matching_decay_1': pattern_matching(1, True, True, decay = 1.001),
    
#     'random_pattern_matching_decay_2': pattern_matching(2, False, False, decay = 1.001),
#     'random_self_pattern_matching_decay_2': pattern_matching(2, False, True, decay = 1.001),
#     'determenistic_pattern_matching_decay_2': pattern_matching(2, True, False, decay = 1.001),
#     'determenistic_self_pattern_matching_decay_2': pattern_matching(2, True, True, decay = 1.001),
    
#     'random_pattern_matching_decay_3': pattern_matching(3, False, False, decay = 1.001),
#     'random_self_pattern_matching_decay_3': pattern_matching(3, False, True, decay = 1.001),
    'determenistic_pattern_matching_decay_3': pattern_matching(3, True, False, decay = 1.001),
    'determenistic_self_pattern_matching_decay_3': pattern_matching(3, True, True, decay = 1.001),
    
#     'random_pattern_matching_decay_4': pattern_matching(4, False, False, decay = 1.001),
#     'random_self_pattern_matching_decay_4': pattern_matching(4, False, True, decay = 1.001),
#     'determenistic_pattern_matching_decay_4': pattern_matching(4, True, False, decay = 1.001),
#     'determenistic_self_pattern_matching_decay_4': pattern_matching(4, True, True, decay = 1.001),
    
#     'random_pattern_matching_decay_5': pattern_matching(5, False, False, decay = 1.001),
#     'random_self_pattern_matching_decay_5': pattern_matching(5, False, True, decay = 1.001),
#     'determenistic_pattern_matching_decay_5': pattern_matching(5, True, False, decay = 1.001),
#     'determenistic_self_pattern_matching_decay_5': pattern_matching(5, True, True, decay = 1.001),
    
#     'random_pattern_matching_decay_6': pattern_matching(6, False, False, decay = 1.001),
#     'random_self_pattern_matching_decay_6': pattern_matching(6, False, True, decay = 1.001),
#     'determenistic_pattern_matching_decay_6': pattern_matching(6, True, False, decay = 1.001),
#     'determenistic_self_pattern_matching_decay_6': pattern_matching(6, True, True, decay = 1.001),
}

history = []
bandit_state = {k:[1,1] for k in agents.keys()}
    
def multi_armed_bandit_agent (observation, configuration):
    
    # bandits' params
    step_size = 3 # how much we increase a and b 
    decay_rate = 1.05 # how much do we decay old historical data
    
    global history, bandit_state
    
    def log_step(step = None, history = None, agent = None, competitorStep = None, file = 'history.csv'):
        if step is None:
            step = np.random.randint(3)
        if history is None:
            history = []
        history.append({'step': step, 'competitorStep': competitorStep, 'agent': agent})
        if file is not None:
            pd.DataFrame(history).to_csv(file, index = False)
        return step
    
    def update_competitor_step(history, competitorStep):
        history[-1]['competitorStep'] = int(competitorStep)
        return history
    
    # load history
    if observation.step == 0:
        pass
    else:
        history = update_competitor_step(history, observation.lastOpponentAction)
        
        # updating bandit_state using the result of the previous step
        # we can update all states even those that were not used
        for name, agent in agents.items():
            agent_step = agent.step(history[:-1])
            bandit_state[name][1] = (bandit_state[name][1] - 1) / decay_rate + 1
            bandit_state[name][0] = (bandit_state[name][0] - 1) / decay_rate + 1
            
            if (history[-1]['competitorStep'] - agent_step) % 3 == 1:
                bandit_state[name][1] += step_size
            elif (history[-1]['competitorStep'] - agent_step) % 3 == 2:
                bandit_state[name][0] += step_size
            else:
                bandit_state[name][0] += step_size/2
                bandit_state[name][1] += step_size/2
            
    # we can use it for analysis later
    with open('bandit.json', 'w') as outfile:
        json.dump(bandit_state, outfile)
    
    
    # generate random number from Beta distribution for each agent and select the most lucky one
    best_proba = -1
    best_agent = None
    for k in bandit_state.keys():
        proba = np.random.beta(bandit_state[k][0],bandit_state[k][1])
        if proba > best_proba:
            best_proba = proba
            best_agent = k
        
    step = agents[best_agent].step(history)
    
    return log_step(step, history, best_agent)

In [ ]:
%%writefile opponent_transition_matrix.py

import numpy as np
import pandas as pd
import random

T = np.zeros((3, 3))
P = np.zeros((3, 3))

# a1 is the action of the opponent 1 step ago
# a2 is the action of the opponent 2 steps ago
a1, a2 = None, None

def transition_agent(observation, configuration):
    global T, P, a1, a2
    if observation.step > 1:
        a1 = observation.lastOpponentAction
        T[a2, a1] += 1
        P = np.divide(T, np.maximum(1, T.sum(axis=1)).reshape(-1, 1))
        a2 = a1
        if np.sum(P[a1, :]) == 1:
            return int((np.random.choice(
                [0, 1, 2],
                p=P[a1, :]
            ) + 1) % 3)
        else:
            return int(np.random.randint(3))
    else:
        if observation.step == 1:
            a2 = observation.lastOpponentAction
        return int(np.random.randint(3))

In [ ]:
%%writefile decision_tree_classifier.py

import numpy as np
import collections
from sklearn.tree import DecisionTreeClassifier

def construct_local_features(rollouts):
    features = np.array([[step % k for step in rollouts['steps']] for k in (2, 3, 5)])
    features = np.append(features, rollouts['steps'])
    features = np.append(features, rollouts['actions'])
    features = np.append(features, rollouts['opp-actions'])
    return features

def construct_global_features(rollouts):
    features = []
    for key in ['actions', 'opp-actions']:
        for i in range(3):
            actions_count = np.mean([r == i for r in rollouts[key]])
            features.append(actions_count)
    
    return np.array(features)

def construct_features(short_stat_rollouts, long_stat_rollouts):
    lf = construct_local_features(short_stat_rollouts)
    gf = construct_global_features(long_stat_rollouts)
    features = np.concatenate([lf, gf])
    return features

def predict_opponent_move(train_data, test_sample):
    classifier = DecisionTreeClassifier(random_state=42)
    classifier.fit(train_data['x'], train_data['y'])
    return classifier.predict(test_sample)

def update_rollouts_hist(rollouts_hist, last_move, opp_last_action):
    rollouts_hist['steps'].append(last_move['step'])
    rollouts_hist['actions'].append(last_move['action'])
    rollouts_hist['opp-actions'].append(opp_last_action)
    return rollouts_hist

def warmup_strategy(observation, configuration):
    global rollouts_hist, last_move
    action = int(np.random.randint(3))
    if observation.step == 0:
        last_move = {'step': 0, 'action': action}
        rollouts_hist = {'steps': [], 'actions': [], 'opp-actions': []}
    else:
        rollouts_hist = update_rollouts_hist(rollouts_hist, last_move, observation.lastOpponentAction)
        last_move = {'step': observation.step, 'action': action}
    return int(action)

def init_training_data(rollouts_hist, k):
    for i in range(len(rollouts_hist['steps']) - k + 1):
        short_stat_rollouts = {key: rollouts_hist[key][i:i+k] for key in rollouts_hist}
        long_stat_rollouts = {key: rollouts_hist[key][:i+k] for key in rollouts_hist}
        features = construct_features(short_stat_rollouts, long_stat_rollouts)        
        data['x'].append(features)
    test_sample = data['x'][-1].reshape(1, -1)
    data['x'] = data['x'][:-1]
    data['y'] = rollouts_hist['opp-actions'][k:]
    return data, test_sample

def agent(observation, configuration):
    # hyperparameters
    k = 5
    min_samples = 25
    global rollouts_hist, last_move, data, test_sample
    if observation.step == 0:
        data = {'x': [], 'y': []}
    # if not enough data -> randomize
    if observation.step <= min_samples + k:
        return warmup_strategy(observation, configuration)
    # update statistics
    rollouts_hist = update_rollouts_hist(rollouts_hist, last_move, observation.lastOpponentAction)
    # update training data
    if len(data['x']) == 0:
        data, test_sample = init_training_data(rollouts_hist, k)
    else:        
        short_stat_rollouts = {key: rollouts_hist[key][-k:] for key in rollouts_hist}
        features = construct_features(short_stat_rollouts, rollouts_hist)
        data['x'].append(test_sample[0])
        data['y'] = rollouts_hist['opp-actions'][k:]
        test_sample = features.reshape(1, -1)
        
    # predict opponents move and choose an action
    next_opp_action_pred = predict_opponent_move(data, test_sample)
    action = int((next_opp_action_pred + 1) % 3)
    last_move = {'step': observation.step, 'action': action}
    return action

In [ ]:
%%writefile statistical_prediction.py

import random
import pydash
from collections import Counter

# Create a small amount of starting history
history = {
    "guess":      [0,1,2],
    "prediction": [0,1,2],
    "expected":   [0,1,2],
    "action":     [0,1,2],
    "opponent":   [0,1],
}
def statistical_prediction_agent(observation, configuration):    
    global history
    actions         = list(range(configuration.signs))  # [0,1,2]
    last_action     = history['action'][-1]
    opponent_action = observation.lastOpponentAction if observation.step > 0 else 2
    
    history['opponent'].append(opponent_action)

    # Make weighted random guess based on the complete move history, weighted towards relative moves based on our last action 
    move_frequency       = Counter(history['opponent'])
    response_frequency   = Counter(zip(history['action'], history['opponent'])) 
    move_weights         = [ move_frequency.get(n,1) + response_frequency.get((last_action,n),1) for n in range(configuration.signs) ] 
    guess                = random.choices( population=actions, weights=move_weights, k=1 )[0]
    
    # Compare our guess to how our opponent actually played
    guess_frequency      = Counter(zip(history['guess'], history['opponent']))
    guess_weights        = [ guess_frequency.get((guess,n),1) for n in range(configuration.signs) ]
    prediction           = random.choices( population=actions, weights=guess_weights, k=1 )[0]

    # Repeat, but based on how many times our prediction was correct
    prediction_frequency = Counter(zip(history['prediction'], history['opponent']))
    prediction_weights   = [ prediction_frequency.get((prediction,n),1) for n in range(configuration.signs) ]
    expected             = random.choices( population=actions, weights=prediction_weights, k=1 )[0]

    # Play the +1 counter move
    action = (expected + 1) % configuration.signs
    
    # Persist state
    history['guess'].append(guess)
    history['prediction'].append(prediction)
    history['expected'].append(expected)
    history['action'].append(action)

    # Print debug information
    print('opponent_action                = ', opponent_action)
    print('move_weights,       guess      = ', move_weights, guess)
    print('guess_weights,      prediction = ', guess_weights, prediction)
    print('prediction_weights, expected   = ', prediction_weights, expected)
    print('action                         = ', action)
    print()
    
    return action

In [ ]:
list_names = [
    "reactionary", 
    "counter_reactionary", 
    "statistical", 
    "nash_equilibrium",
    "markov_agent", 
    "memory_patterns", 
    "multi_armed_bandit",
    "opponent_transition_matrix",
    "decision_tree_classifier",
    "statistical_prediction",
]

list_agents = [agent_name + ".py" for agent_name in list_names]

# initate output format
output = {}
for agent in list_agents:
    output[agent] = {
        'win'  : 0,
        'loss' : 0,
        'tie'  : 0
    }

## Multiprocessing Using ray

In [ ]:
from ray.util.multiprocessing import Pool as rayPool
import pandas as pd
import numpy as np
from kaggle_environments import make, evaluate
import tqdm

In [ ]:
# set number of times to play. If there are 5 agents and x = 4, there are going to be total of 5*(5-1)*4 = 80 plays.
x = 3
list_of_games = []
for times in range(x):
    for i in list_agents:
        for j in list_agents:
            if i != j:
                list_of_games.append([i,j])
list_of_games[:10]

In [ ]:
# function to return score
def get_result(agents):
    output = evaluate(
                "rps", 
                [agents[0], agents[1]], 
                configuration={"episodeSteps": 1000})
    return agents[0], agents[1], output[0]

In [ ]:
results = []

# multiprocessing
pool = rayPool()
for content in tqdm.tqdm(pool.imap_unordered(get_result, list_of_games), total = len(list_of_games)):
    results.append(content)
pool.close()
    
# generate final output
for result in results:
    if abs(result[2][0]) < 20:
        output[result[0]]['tie']  += 1
        output[result[1]]['tie']  += 1
    elif result[2][0] >= 20:
        output[result[0]]['win']  += 1
        output[result[1]]['loss'] += 1
    elif result[2][0] <= -20:
        output[result[0]]['loss'] += 1
        output[result[1]]['win']  += 1


#### Check overall win-loss records

In [ ]:
pd.DataFrame.from_dict(output).T[['win','loss', 'tie']].sort_values('win', ascending=False)

#### Check results between two agents

In [ ]:
def show_me(agent1, agent2):
    w = 0
    l = 0
    t = 0
    for i in results:
        if i[0] == agent1 and i[1] == agent2:
            if i[2][0] >= 20:
                w += 1
            elif i[2][0] <= -20:
                l += 1
            else:
                t += 1
        elif i[0] == agent2 and i[1] == agent1:
            if i[2][0] >= 20:
                l += 1
            elif i[2][0] <= -20:
                w += 1
            else:
                t += 1           
    print('win: ' + str(w) + ' loss: ' + str(l) + ' tie: ' + str(t) + '  -  ' + agent1 + ' vs ' + agent2)
    
for i in list_agents:
    for j in list_agents:
        if i != j:
            show_me(i, j)
    print('')

#### Check individual win-loss records

In [ ]:
individual_output = {}

for i in range(len(list_agents)):
    for j in range(i+1, len(list_agents)):
        filtered_results = [x for x in results if (list_agents[i] in x) and (list_agents[j] in x)]
        individual_output[list_agents[i] + ' vs ' + list_agents[j]] = {
        'agent1': list_agents[i],
        'agent2': list_agents[j],
        'win'   : 0,
        'loss'  : 0,
        'tie'   : 0            
        }
        individual_output[list_agents[i] + ' vs ' + list_agents[j]]['win']  += sum((1 for x in filtered_results if x[0] == list_agents[i] and x[2][0]>=20))
        individual_output[list_agents[i] + ' vs ' + list_agents[j]]['loss'] += sum((1 for x in filtered_results if x[0] == list_agents[i] and x[2][0]<=-20))
        individual_output[list_agents[i] + ' vs ' + list_agents[j]]['tie']  += sum((1 for x in filtered_results if x[0] == list_agents[i] and abs(x[2][0])<20))
        individual_output[list_agents[i] + ' vs ' + list_agents[j]]['win']  += sum((1 for x in filtered_results if x[0] == list_agents[j] and x[2][0]<=-20))
        individual_output[list_agents[i] + ' vs ' + list_agents[j]]['loss'] += sum((1 for x in filtered_results if x[0] == list_agents[j] and x[2][0]>=20))
        individual_output[list_agents[i] + ' vs ' + list_agents[j]]['tie']  += sum((1 for x in filtered_results if x[0] == list_agents[j] and abs(x[2][0])<20))
pd.DataFrame.from_dict(individual_output).T[['agent1', 'agent2','win','loss', 'tie']].reset_index(drop=True)

## Multiprocessing Using multiprocessing

In [ ]:
from multiprocessing import Pool as mpPool

In [ ]:
# initate output format
output = {}
for agent in list_agents:
    output[agent] = {
        'win'  : 0,
        'loss' : 0,
        'tie'  : 0
    }

In [ ]:
results = []

# multiprocessing
pool = mpPool()
for content in tqdm.tqdm(pool.imap_unordered(get_result, list_of_games), total = len(list_of_games)):
    results.append(content)
pool.close()
    
# generate final output
for result in results:
    if abs(result[2][0]) < 20:
        output[result[0]]['tie']  += 1
        output[result[1]]['tie']  += 1
    elif result[2][0] >= 20:
        output[result[0]]['win']  += 1
        output[result[1]]['loss'] += 1
    elif result[2][0] <= -20:
        output[result[0]]['loss'] += 1
        output[result[1]]['win']  += 1


#### Check overall win-loss records

In [ ]:
pd.DataFrame.from_dict(output).T[['win','loss', 'tie']].sort_values('win', ascending=False)

#### Check results between two agents

In [ ]:
def show_me(agent1, agent2):
    w = 0
    l = 0
    t = 0
    for i in results:
        if i[0] == agent1 and i[1] == agent2:
            if i[2][0] >= 20:
                w += 1
            elif i[2][0] <= -20:
                l += 1
            else:
                t += 1
        elif i[0] == agent2 and i[1] == agent1:
            if i[2][0] >= 20:
                l += 1
            elif i[2][0] <= -20:
                w += 1
            else:
                t += 1           
    print('win: ' + str(w) + ' loss: ' + str(l) + ' tie: ' + str(t) + '  -  ' + agent1 + ' vs ' + agent2)
    
for i in list_agents:
    for j in list_agents:
        if i != j:
            show_me(i, j)
    print('')

#### Check individual win-loss records

In [ ]:
individual_output = {}

for i in range(len(list_agents)):
    for j in range(i+1, len(list_agents)):
        filtered_results = [x for x in results if (list_agents[i] in x) and (list_agents[j] in x)]
        individual_output[list_agents[i] + ' vs ' + list_agents[j]] = {
        'agent1': list_agents[i],
        'agent2': list_agents[j],
        'win'   : 0,
        'loss'  : 0,
        'tie'   : 0            
        }
        individual_output[list_agents[i] + ' vs ' + list_agents[j]]['win']  += sum((1 for x in filtered_results if x[0] == list_agents[i] and x[2][0]>=20))
        individual_output[list_agents[i] + ' vs ' + list_agents[j]]['loss'] += sum((1 for x in filtered_results if x[0] == list_agents[i] and x[2][0]<=-20))
        individual_output[list_agents[i] + ' vs ' + list_agents[j]]['tie']  += sum((1 for x in filtered_results if x[0] == list_agents[i] and abs(x[2][0])<20))
        individual_output[list_agents[i] + ' vs ' + list_agents[j]]['win']  += sum((1 for x in filtered_results if x[0] == list_agents[j] and x[2][0]<=-20))
        individual_output[list_agents[i] + ' vs ' + list_agents[j]]['loss'] += sum((1 for x in filtered_results if x[0] == list_agents[j] and x[2][0]>=20))
        individual_output[list_agents[i] + ' vs ' + list_agents[j]]['tie']  += sum((1 for x in filtered_results if x[0] == list_agents[j] and abs(x[2][0])<20))
pd.DataFrame.from_dict(individual_output).T[['agent1', 'agent2','win','loss', 'tie']].reset_index(drop=True)

## Without multiprocessing

In [ ]:
# initate output format
output = {}
for agent in list_agents:
    output[agent] = {
        'win'  : 0,
        'loss' : 0,
        'tie'  : 0
    }

In [ ]:
results = []

for content in tqdm.tqdm(list_of_games, total = len(list_of_games)):
    results.append(get_result(content))
pool.close()
    
# generate final output
for result in results:
    if abs(result[2][0]) < 20:
        output[result[0]]['tie']  += 1
        output[result[1]]['tie']  += 1
    elif result[2][0] >= 20:
        output[result[0]]['win']  += 1
        output[result[1]]['loss'] += 1
    elif result[2][0] <= -20:
        output[result[0]]['loss'] += 1
        output[result[1]]['win']  += 1


#### Check overall win-loss records

In [ ]:
pd.DataFrame.from_dict(output).T[['win','loss', 'tie']].sort_values('win', ascending=False)

#### Check results between two agents

In [ ]:
def show_me(agent1, agent2):
    w = 0
    l = 0
    t = 0
    for i in results:
        if i[0] == agent1 and i[1] == agent2:
            if i[2][0] >= 20:
                w += 1
            elif i[2][0] <= -20:
                l += 1
            else:
                t += 1
        elif i[0] == agent2 and i[1] == agent1:
            if i[2][0] >= 20:
                l += 1
            elif i[2][0] <= -20:
                w += 1
            else:
                t += 1           
    print('win: ' + str(w) + ' loss: ' + str(l) + ' tie: ' + str(t) + '  -  ' + agent1 + ' vs ' + agent2)
    
for i in list_agents:
    for j in list_agents:
        if i != j:
            show_me(i, j)
    print('')

#### Check individual win-loss records

In [ ]:
individual_output = {}

for i in range(len(list_agents)):
    for j in range(i+1, len(list_agents)):
        filtered_results = [x for x in results if (list_agents[i] in x) and (list_agents[j] in x)]
        individual_output[list_agents[i] + ' vs ' + list_agents[j]] = {
        'agent1': list_agents[i],
        'agent2': list_agents[j],
        'win'   : 0,
        'loss'  : 0,
        'tie'   : 0            
        }
        individual_output[list_agents[i] + ' vs ' + list_agents[j]]['win']  += sum((1 for x in filtered_results if x[0] == list_agents[i] and x[2][0]>=20))
        individual_output[list_agents[i] + ' vs ' + list_agents[j]]['loss'] += sum((1 for x in filtered_results if x[0] == list_agents[i] and x[2][0]<=-20))
        individual_output[list_agents[i] + ' vs ' + list_agents[j]]['tie']  += sum((1 for x in filtered_results if x[0] == list_agents[i] and abs(x[2][0])<20))
        individual_output[list_agents[i] + ' vs ' + list_agents[j]]['win']  += sum((1 for x in filtered_results if x[0] == list_agents[j] and x[2][0]<=-20))
        individual_output[list_agents[i] + ' vs ' + list_agents[j]]['loss'] += sum((1 for x in filtered_results if x[0] == list_agents[j] and x[2][0]>=20))
        individual_output[list_agents[i] + ' vs ' + list_agents[j]]['tie']  += sum((1 for x in filtered_results if x[0] == list_agents[j] and abs(x[2][0])<20))
pd.DataFrame.from_dict(individual_output).T[['agent1', 'agent2','win','loss', 'tie']].reset_index(drop=True)